# Desafio 2
## Marcelo Leopoldo e Silva de Carvalho Filho - RM 342876

### Obtendo informações importantes acerca do dataset

Vamos utilizar o dataset breast_cancer_data.csv para fazer uma análise exploratória cujo objetivo é responder a alguns questionamentos que surgiram a respeito desse conjunto.

Vamos começar lendo e entendendo nossos dados:

In [2]:
import pandas as pd
import numpy as np

In [4]:
colunas = ['id','clump_thickness','cell_size_uniformity','cell_shape_uniformity','marginal_adhesion','single_ep_cell_size',
          'bare_nuclei','bland_chromatin','normal_nucleoli','mitoses','class','doctor_name']

data = pd.read_csv('bases/breast_cancer_data.csv',names=colunas)
data.head()

,id,clump_thickness,cell_size_uniformity,cell_shape_uniformity,marginal_adhesion,single_ep_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class,doctor_name
0,1000025,5.0,1.0,1,1,2,1,3.0,1.0,1,benign,Dr. Doe
1,1002945,5.0,4.0,4,5,7,10,3.0,2.0,1,benign,Dr. Smith
2,1015425,3.0,1.0,1,1,2,2,3.0,1.0,1,benign,Dr. Lee
3,1016277,6.0,8.0,8,1,3,4,3.0,7.0,1,benign,Dr. Smith
4,1017023,4.0,1.0,1,3,2,1,3.0,1.0,1,benign,Dr. Wong


In [16]:
# TODO: entendimento básico dos dados - preencha as lacunas no código a fim de obter o resultado desejado

# quantas linhas e quantas colunas existem no dataset?
print('formato: ', data.shape, '\n')

# quais os tipos de dados de cada coluna?
print('tipos de dados: ', data.dtypes,'\n')

# quais as principais estatísticas dos dados numéricos do dataset?
print('principais estatísticas: ', data.describe(), '\n')

formato:  (699, 12) 

tipos de dados:  id                         int64
clump_thickness          float64
cell_size_uniformity     float64
cell_shape_uniformity      int64
marginal_adhesion          int64
single_ep_cell_size        int64
bare_nuclei               object
bland_chromatin          float64
normal_nucleoli          float64
mitoses                    int64
class                     object
doctor_name               object
dtype: object 

principais estatísticas:                   id  clump_thickness  cell_size_uniformity  \
count  6.990000e+02       698.000000            698.000000   
mean   1.071704e+06         4.416905              3.137536   
std    6.170957e+05         2.817673              3.052575   
min    6.163400e+04         1.000000              1.000000   
25%    8.706885e+05         2.000000              1.000000   
50%    1.171710e+06         4.000000              1.000000   
75%    1.238298e+06         6.000000              5.000000   
max    1.345435e+07        

In [20]:
# TODO: obtendo informações sobre o dataset

# retorne uma Serie que mostre qual a classificação (maligno e benigno) que cada médico deu para cada amostra: 
# Exemplo de retorno
# class      doctor_name
# benign   Dr. Doe        127
#          Dr. Lee        121
#          Dr. Smith      102
#          Dr. Wong       108
# Código:
data.groupby(['class', 'doctor_name'])['cell_size_uniformity'].mean()


class      doctor_name
benign     Dr. Doe        1.253968
           Dr. Lee        1.256198
           Dr. Smith      1.382353
           Dr. Wong       1.435185
malignant  Dr. Doe        6.206897
           Dr. Lee        6.366667
           Dr. Smith      6.918919
           Dr. Wong       6.734694
Name: cell_size_uniformity, dtype: float64

In [61]:
# TODO: obtendo informações sobre o dataset

# Use o método select_dtypes() para selecionar apenas colunas do tipo float64 e int64. Armazene na variável data_num
data_num = data.select_dtypes(['float64', 'int64'])
# Encontre as n variáveis que possuem um valor absoluto de correlação maior que 0.5 com a variável clump_thickness, excluindo ela mesma 
# retorne os valores de forma descendente
# sempre utilize a variável data_num para receber o resultado de suas operações

data_num = data_num.loc[:, data_num.columns != 'clump_thickness'].corrwith(data_num['clump_thickness'])
data_num = data_num.loc[data_num > 0.5].sort_values(ascending=False)

print("Existem {} variáveis correlacionadas com clump_thickness:\n{}".format(len(data_num), data_num))

Existem 5 variáveis correlacionadas com clump_thickness:
cell_shape_uniformity    0.654632
cell_size_uniformity     0.644636
bland_chromatin          0.558924
normal_nucleoli          0.535773
single_ep_cell_size      0.522108
dtype: float64


In [82]:
# TODO: obtendo informações sobre o dataset

# qual o tamanho médio das células para cada tipo de tumor?
mean_size = data.groupby('class')['clump_thickness'].mean()
print('Tamanho médio do tipo benigno: ',mean_size['benign'],'\n')
print('Tamanho médio do tipo maligno: ',mean_size['malignant'],'\n')

# qual o número máximo de mitoses em tumores benignos encontrados pelo Dr. Wong?
max_mitoses_wong = data.loc[(data['doctor_name'] == 'Dr. Wong') & (data['class'] == 'benign')]['mitoses'].max()
print('Número máximo de mitoses em tumores benignos: ', max_mitoses_wong)

# e qual o número máximo de mitoses em tumores malignos encontrados pelo mesmo médico?
max_malignant_mitoses = data.loc[data['class'] == 'malignant']['mitoses'].max()
print('\nNúmero máximo de mitoses em tumores malignos: ',max_malignant_mitoses)

Tamanho médio do tipo benigno:  2.9563318777292578 

Tamanho médio do tipo maligno:  7.204166666666667 

Número máximo de mitoses em tumores benignos:  3

Número máximo de mitoses em tumores malignos:  10


In [136]:
# TODO: filtrando os dados

# Existem dois pacientes (id) que aparecem mais que duas vezes no dataset. Retorne uma série ordenada de forma descendente que mostre o
# id do paciente bem como a quantidade de vezes que ele aparece repetido no dataset
repeated_patients = data[data.duplicated(['id'], keep=False)].groupby('id')['id'].count()
# Elimine esses dois pacientes do dataframe de modo que este contenha apenas pacientes que apareceram no máximo duas vezes
filtered_patients = repeated_patients[repeated_patients > 2]
filtered_data = data[~data['id'].isin(filtered_patients.index)] 

print("data size: ", data.shape)
print("patients that repeats more than two times:", filtered_patients)
print("filtered data size:", filtered_data.shape)

data size:  (699, 12)
patients that repeats more than two times: id
1182404    6
1198641    3
1276091    5
Name: id, dtype: int64
filtered data size: (685, 12)
